In [48]:
# EXTENSIONS
%load_ext autoreload
%autoreload 2

In [116]:
# IMPORTS
import os
from glob import glob
import torch
import pandas as pd
import numpy as np
import rasterio
from data_director import DataDirector
from model import RoCNN


In [ ]:
# Compute min and max values per band across all images.
file_paths = glob("../data/db/___trn_val_tst/processed/*.tif")
tensor_min = None
tensor_max = None
for p in file_paths:
    img = rasterio.open(p)
    img = torch.tensor(img.read().astype('float32'))
    img_min = torch.min(torch.min(img, dim=1).values, dim=1).values
    img_max = torch.max(torch.max(img, dim=1).values, dim=1).values
    if tensor_min is None: tensor_min = img_min
    else: tensor_min = torch.where((img_min < tensor_min), 
                                   img_min, tensor_min)
    if tensor_max is None: tensor_max = img_max
    else: tensor_max = torch.where((img_max > tensor_max), 
                                   img_max, tensor_max)
print("tensor_min:", tensor_min)
print("tensor_max:", tensor_max)

tensor_min: tensor([37.6562, 18.8790, 24.0802, 14.8663,  6.4967,  5.2562])
tensor_max: tensor([186.6202, 129.2332, 165.8419,  92.4461,  96.8614,  55.0911])


In [78]:
# Get image shape.
file_paths = glob("../data/db/___trn_val_tst/processed/*.tif")
img_shapes = set()
for p in file_paths:
    img = rasterio.open(p)
    img = torch.tensor(img.read().astype('float32'))
    img_shape = img.cpu().numpy().shape
    img_shapes.add(img_shape)
print("Image shapes found:", img_shapes)

Image shapes found: {(6, 64, 62), (6, 64, 64)}


In [149]:
dd = DataDirector(dir_data="../data/db/___trn_val_tst/processed",
                  split_pc_trn=0.8,
                  split_pc_val=0.1,
                  batch_size=16)

Finding min and max values per band ... Done :)
Initializing DataLoaders ... len(self._paths) = 416
len(self._paths) = 52
len(self._paths) = 53
Done :)


In [172]:
y, x = list(dd.dataloader["trn"])[0]
if y.ndim == 1: 
    y = y.unsqueeze(1) # make [B,1]
    y = y.to(torch.float32)

In [159]:
m = RoCNN()

In [174]:
p = m(x)

In [190]:
bce_loss = torch.nn.BCELoss(reduction='none')

In [191]:
loss = bce_loss(p, y) # Compute loss.

In [197]:
x.size(0)

16

In [ ]:
p_bin = (p >= 0.5).float()

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])